In [1]:
import os
import re
import yaml
import json
import ast
import praw
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

from textwrap import dedent
from reddit_helper import *    

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

from pydantic import BaseModel
from datetime import datetime, timedelta

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
api_file_path = 'api_keys.json'
with open(api_file_path, 'r') as file:
    api_keys = json.load(file)        
openai_gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])
openai_gpt4 = ChatOpenAI(model_name="gpt-4", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])        
llama31_8b = Ollama(model='llama3.1:8b',)
gemma2_9b = Ollama(model='gemma2:9b',)

In [3]:
cfg_file_path = 'casaai_config.yaml'
agent_cfg_path = 'agents.yaml'
task_cfg_path = 'tasks.yaml'

with open(cfg_file_path, 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)
product_long = cfg.get('product_long_description', '')      
product_short = cfg.get('product_short_description', '')
product_name = cfg.get('product_short_description', '')
product_url = cfg.get('product_url', '')
domain = cfg.get('domain')
broad_keywords = cfg.get('broad_keywords', [])
output_format_1 = cfg.get('output_format_1')

with open(agent_cfg_path, 'r') as yaml_file:
    agent_cfg_data = yaml.safe_load(yaml_file)

with open(task_cfg_path, 'r') as yaml_file:
    task_cfg_data = yaml.safe_load(yaml_file)

In [4]:
tip_text = "If you do your BEST WORK, I'll give you a $100 commission!"

In [5]:
def re_decision_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'decision':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, decision = matches[0]
    else:
        comment_id = "na"
        decision = "discard"

    if "justification" in input_string:
        justification = input_string.split("justification")[-1]
    else:
        justification = "Not available"
    
    return comment_id, decision, justification

In [6]:
def re_score_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'score':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, score = matches[0]
    else:
        comment_id = "na"
        score = 0

    if "justification" in input_string:
        justification = input_string.split("justification")[-1]
    else:
        justification = "Not available"
    
    return comment_id, score, justification

In [7]:
def re_response_string(input_string):

    if "responz" in input_string:
        response = input_string.split("responz")[-1]
    else:
        response = "Not available"
    
    return response

In [8]:
def get_calendar_date(unix_timestamp):
    date_obj = datetime.utcfromtimestamp(unix_timestamp)
    readable_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
    return readable_date

In [9]:
def prep_decision_output(decision_lst, old_comments):
    decision_analysis_lst = []
    decision_bad_list = []
    tot_comments = 0
    bad_cnt = 0
    decision_relevant_json_list = []
    decision_discarded_json_list = []
    for item in decision_lst:  
        sep = "^"
        for comment in item:
            tot_comments += 1
            try:
                comm_key = list(comment.keys())[0]
                comm_id  = comment[comm_key].strip()
                if comm_id in comment_dict_list:                 
                    parent_id = comment_dict[comm_id]["parent_id"]
                    if parent_id is None:
                        parent_id = 'NA'
                    decision = comment['decision']
                    justify  = comment["justification"]
                    comment_text = comment_dict[comm_id]["text"]
                    text_str = sep + comm_id + sep + parent_id + sep + decision + sep + justify + sep + comment_text + sep
                    if comment['decision'] == 'relevant':
                        decision_relevant_json_list.append(comment)
                    else:
                        decision_discarded_json_list.append(comment)
                    decision_analysis_lst.append(text_str)            
                else: 
                    print(f'Comment ID not found : {comm_id}')
                    bad_cnt += 1
                    decision_bad_list.append(comm_id)
            except:
                print(f'comment key error : {comment}')
                bad_cnt += 1
    print(f'relevant : {len(decision_relevant_json_list)} discarded : {len(decision_discarded_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')
    
    old_comment_list = []
    for comment in old_comments:  
        sep = "^"
        comm_id  = comment["comment_id"]
        parent_id = comment["parent_id"]
        if parent_id is None:
            parent_id = 'NA'        
        comment_text  = comment["text"]
        created_date = get_calendar_date(comment['created_utc'])
        age = str(comment["age"])
        text_str = sep + comm_id + sep + parent_id + sep + age + sep + created_date + sep + comment_text + sep
        old_comment_list.append(text_str)      
    print(f'Old comments already filtered out : {len(old_comments)}')
    
    output = (decision_relevant_json_list, decision_discarded_json_list, decision_analysis_lst, old_comment_list)
    return output

In [10]:
def prep_score_output(score_result_lst):
    analysis_lst = []
    bad_list = []
    tot_comments = 0
    bad_cnt = 0
    good_json_list = []
    sep = "^"
    for comment in score_result_lst:
        tot_comments += 1
        try:
            comm_key = list(comment.keys())[0]
            comm_id  = comment[comm_key].strip()
            
            if comm_id in comment_dict_list:  
                parent_id = comment_dict[comm_id]["parent_id"]
                if parent_id is None:
                    parent_id = 'NA'
                score = comment['score']
                age = comment_dict[comm_id]["age"]
                created = comment_dict[comm_id]["created"]
                justify  = comment["justification"]
                comment_text = comment_dict[comm_id]["text"]
                text_str = sep + comm_id + sep + parent_id + sep + str(score) + sep + str(age) + sep + created + sep + justify \
                            + sep + comment_text + sep
                analysis_lst.append(text_str) 
                good_json_list.append(comm_id)
            else: 
                print(f'Comment ID not found : {comm_id}')
                bad_cnt += 1
                bad_list.append(comm_id)
        except:
            print(f'comment key error : {comment}')
            bad_cnt += 1
    print(f'relevant : {len(good_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')
    output = (good_json_list, bad_list, analysis_lst)
    return output

In [11]:
def prep_response_output(response_result_lst):
    analysis_lst = []
    bad_list = []
    tot_comments = 0
    bad_cnt = 0
    good_json_list = []
    sep = "^"
    for comment in response_result_lst:
        tot_comments += 1
        try:
            comm_id  = comment['comm_id'].strip()
            
            if comm_id in comment_dict_list:  
                parent_id = comment_dict[comm_id]["parent_id"]
                if parent_id is None:
                    parent_id = 'NA'
                age = comment_dict[comm_id]["age"]
                created = comment_dict[comm_id]["created"]
                response  = comment["responz"]
                comment_text = comment_dict[comm_id]["text"]
                text_str = sep + comm_id + sep + parent_id + sep + str(age) + sep + created + sep + response \
                            + sep + comment_text + sep
                analysis_lst.append(text_str) 
                good_json_list.append(comm_id)
            else: 
                print(f'Comment ID not found : {comm_id}')
                bad_cnt += 1
                bad_list.append(comm_id)
        except:
            print(f'comment key error : {comment}')
            bad_cnt += 1
    print(f'relevant : {len(good_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')
    output = (good_json_list, bad_list, analysis_lst)
    return output

In [12]:
backstory = agent_cfg_data['content_filter_analyst']['backstory'] 
goal = agent_cfg_data['content_filter_analyst']['goal']
role = agent_cfg_data['content_filter_analyst']['role'] 
content_filter_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [13]:
class DecisionOutput(BaseModel):
    comment_id: str
    decision: str
    justification: str

In [14]:
description = task_cfg_data['content_filter_task']['description']
expected_out = task_cfg_data['content_filter_task']['expected_out'] 
        
content_filter_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=DecisionOutput,
                            agent=content_filter_analyst,
                            )

In [15]:
response_creation_crew = Crew(
    agents=[content_filter_analyst,],
    tasks=[content_filter_task,],
    verbose=False,
)

In [16]:
reddit_posts, reddit_post_ids = fetch_reddit_test()
condensed_reddit_data, unique_post_ids, unique_comment_ids = condense_data(reddit_posts, reddit_post_ids)

post_cnt: 0 + 20 = 20
comm_cnt: 0 + 696 = 696
cond_cnt: 716 = 20 + 696
Cross_ck: 20 = 20


In [17]:
comment_dict = {}
tot_data = 0
for idx1, item in enumerate(condensed_reddit_data):
    for idx2, data in enumerate(item):
        tot_data += 1
        c_id = data['comment_id']
        p_id = data['parent_id']
        text = data['text']
        created_date = get_calendar_date(data['created_utc'])
        age = data['age']
        comment_dict[c_id] = {'comment_id': c_id, 'parent_id': p_id, 'text': text, 'age': age, 'created': created_date}
    print(idx1, ':', idx2)
comment_dict_list = list(comment_dict.keys())
print(len(comment_dict), '=', tot_data, len(comment_dict_list))

0 : 169
1 : 108
2 : 31
3 : 57
4 : 0
5 : 33
6 : 60
7 : 67
8 : 1
9 : 34
10 : 0
11 : 79
12 : 1
13 : 0
14 : 7
15 : 2
16 : 0
17 : 9
18 : 34
19 : 4
716 = 716 716


In [18]:
def get_data_details(comm_id):
    done = 0
    data_details = []
    while done==0:
        if comm_id in comment_dict:
            details = comment_dict[comm_id]
            data_details.append(details)
            comm_id = details['parent_id']
        else:
            done = 1
    return data_details

In [19]:
decision_lst = []
old_comments = []
age_limit = 3
for idx, reddit_data_item in enumerate(condensed_reddit_data):
    comment_lst = []
    for idx2, comment_data in enumerate(reddit_data_item):
        if comment_data['age'] < age_limit:
            comm_id = comment_data['comment_id']  
            comment_text = comment_data['text']
            print(f'STARTING {idx} - {idx2} - {comm_id}')   
            input_dict = {"comment_id": comm_id,
                          "input_data": comment_text,
                          "product_long": product_long,
                          "product_short": product_short,
                          "domain": domain,
                          "broad_keywords": broad_keywords,
                          "output_format":output_format_1, 
                          "tip_text":tip_text}
            decision_result = response_creation_crew.kickoff(inputs=input_dict)
            try:
                json_out = ast.literal_eval(decision_result.json)
            except:
                print('Exception in json - trying re')
                _, d, j = re_decision_string(decision_result.raw)            
                json_out = {'comm_id': comm_id, 'decision': str(d), 'justification': str(j)}
            comment_lst.append(json_out)
        else:
            old_comments.append(comment_data)
    decision_lst.append(comment_lst)

STARTING 2 - 0 - 1baop85
STARTING 2 - 1 - ku4kjo6
STARTING 2 - 2 - ku47yt2
STARTING 2 - 3 - ku3v9wy
STARTING 2 - 4 - ku465r4
STARTING 2 - 5 - ku4gy4w
 Pydantic validation error: 1 validation error for DecisionOutput
comment_id
  Field required [type=missing, input_value={'comment id': 'ku4gy4w',...nt to interior design.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing. Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for DecisionOutput
comment_id
  Field required [type=missing, input_value={'comment id': 'ku4gy4w',...nt to interior design.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing. The JSON structure doesn't match the expected model. Attempting alternative conversion method.
STARTING 2 - 6 - ku4zdyy
STARTING 2 - 7 - ku5spxg
STARTING 2 - 8 - ku4187y
STARTING 2 - 9 - ku4iico
STARTING 2 - 10 - ku4girb
STARTING 2 - 11 - ku47o9a
STARTING 2 - 12 - ku6hp46
STAR

In [20]:
decision_out = prep_decision_output(decision_lst, old_comments)

comment key error : {'ku5l2mj': {'decision': 'relevant', 'justification': 'Seeks design advice on kitchen elements.'}}
Comment ID not found : string
Comment ID not found : krclyt
Comment ID not found : example_id
Comment ID not found : Comment 1
Comment ID not found : iyv709
relevant : 75 discarded : 115 bad_cnt: 6 total: 196
Old comments already filtered out : 520


In [21]:
for item in decision_out:
    print(len(item))
content_filter_relevant, content_filter_discard, content_filter_analysis, content_filter_old = decision_out
content_filter_relevant[0]

75
115
190
520


{'comment_id': 'ku4kjo6',
 'decision': 'relevant',
 'justification': 'Comment seeks advice on making a kitchen functional and usable.'}

In [22]:
with open('decision_result_v6_0902A.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_filter_analysis:
        file.write(item + "\n")  # Add a newline character after each string
with open('decision_old_v6_0902A.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_filter_old:
        file.write(item + "\n")  # Add a newline character after each string

print(f"Files has been written")

with open('decision_result_v6_0902A.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(f"Files Reloaded")

Files has been written
Files Reloaded


In [23]:
backstory = agent_cfg_data['content_scoring_analyst']['backstory'] 
goal = agent_cfg_data['content_scoring_analyst']['goal']
role = agent_cfg_data['content_scoring_analyst']['role'] 
content_scoring_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [24]:
class ScoreOutput(BaseModel):
    comment_id: str
    score: float
    justification: str

In [25]:
description = task_cfg_data['content_scoring_task']['description']
expected_out = task_cfg_data['content_scoring_task']['expected_out'] 
        
content_scoring_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=ScoreOutput,
                            agent=content_filter_analyst,
                            )

In [26]:
score_creation_crew = Crew(
    agents=[content_scoring_analyst,],
    tasks=[content_scoring_task,],
    verbose=False,
)

2024-09-02 11:10:14,721 - 140078363072320 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [27]:
score_result_lst = []
for idx, comment_data in enumerate(content_filter_relevant):     
    comm_id = comment_data['comment_id']        
    print(f'STARTING {idx} - {comm_id}')
    data_details = get_data_details(comm_id)      
    input_dict = {"comment_id": comm_id,
                  "input_data": data_details,
                  "product_long": product_long,
                  "product_short": product_short,
                  "domain": domain,
                  "product_name": product_name,
                  "tip_text":tip_text}
    scoring_result = score_creation_crew.kickoff(inputs=input_dict)
    try:
        json_out = ast.literal_eval(scoring_result.json)
    except:
        print('Exception in json - trying re')
        _, s, j = re_score_string(scoring_result.raw)            
        json_out = {'comm_id': comm_id, 'score': float(s), 'justification': str(j)}            
    score_result_lst.append(json_out)

STARTING 0 - ku4kjo6
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 1 - ku47yt2
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 2 - ku4gy4w
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 3 - ku6xgy6
 Error parsing JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1). Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for ScoreOutput
  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value="{'comment_id': 'ku6xgy6'..., average engagement.'}", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid. The JSON structure doesn't match the expected model. Attempting alternative conversion method.
STARTING 4 - kubtemt
 Error parsing JSON: Expecting property name enclosed in double quo

In [28]:
score_result_lst[0], len(score_result_lst)

({'comment_id': 'ku4kjo6',
  'score': 8.2,
  'justification': "Expresses desire for practical, functional design aligning with CasaAI's capabilities."},
 75)

In [29]:
thresh_list = [7.0, 7.5, 8.0, 8.5, 9.0, 9.5]

for thresh in thresh_list:
    thresh_up = 0
    for item in score_result_lst:
        if item['score'] >= thresh:
            thresh_up += 1
    print(thresh, ':', thresh_up, '/', len(score_result_lst))

7.0 : 39 / 75
7.5 : 29 / 75
8.0 : 15 / 75
8.5 : 12 / 75
9.0 : 2 / 75
9.5 : 0 / 75


In [30]:
score_out = prep_score_output(score_result_lst)
content_score_good, content_score_bad, content_score_analysis = score_out
print(len(content_score_good), len(content_score_analysis))

relevant : 75 bad_cnt: 0 total: 75
75 75


In [31]:
with open('score_result_v6_0902A.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_score_analysis:
        file.write(item + "\n")  # Add a newline character after each string

print(f"File has been written")

with open('score_result_v6_0902A.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

File has been written
118


In [32]:
# For each post-comment combo select the post-comments whose score >= 7
filter_thresh = 7.0
filtered_comment_ids = []
for item in score_result_lst:
    if item['score'] >= filter_thresh:
        filtered_comment_ids.append(item['comment_id'])
print(f'len(filtered_comment_ids) : {len(filtered_comment_ids)}')

len(filtered_comment_ids) : 39


In [33]:
backstory = agent_cfg_data['response_writer']['backstory'] 
goal = agent_cfg_data['response_writer']['goal']
role = agent_cfg_data['response_writer']['role'] 
response_writer = Agent(
                        role=role,
                        goal=goal,
                        backstory=backstory,
                        allow_delegation=False,
                        verbose=True,
                        llm=gemma2_9b,
                        )

In [34]:
backstory = agent_cfg_data['response_reviewer']['backstory'] 
goal = agent_cfg_data['response_reviewer']['goal']
role = agent_cfg_data['response_reviewer']['role'] 
response_reviewer = Agent(
                        role=role,
                        goal=goal,
                        backstory=backstory,
                        allow_delegation=True,
                        verbose=True,
                        llm=gemma2_9b,
                        )

In [35]:
class ResponseOutput(BaseModel):
    responz: str

In [36]:
description = task_cfg_data['response_writing_task']['description']
expected_out = task_cfg_data['response_writing_task']['expected_out'] 
        
response_writing_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=ResponseOutput,
                            agent=response_writer,
                            )

In [37]:
description = task_cfg_data['response_review_task']['description']
expected_out = task_cfg_data['response_review_task']['expected_out'] 
        
response_review_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=ResponseOutput,
                            agent=response_reviewer,
                            )

In [38]:
response_creation_crew = Crew(
    agents=[response_writer, response_reviewer],
    tasks=[response_writing_task, response_review_task],
    verbose=True,
)

2024-09-02 11:17:54,232 - 140078363072320 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [39]:
response_result_lst = []
for idx, comment_id in enumerate(filtered_comment_ids):           
    print(f'STARTING {idx} - {comment_id}')
    input_data = comment_dict[comment_id]
    data_details = get_data_details(comment_id)      
    input_dict = {"comment_id": comment_id,
                  "input_data": input_data,
                  "context": data_details,
                  "product_long": product_long,
                  "product_short": product_short,
                  "domain": domain,
                  "product_name": product_name,
                  "product_url": product_url,
                  "tip_text":tip_text}
    response_result = response_creation_crew.kickoff(inputs=input_dict)
    try:
        json_out = ast.literal_eval(response_result.json)
        json_out['comm_id'] = comment_id
    except:
        print('Exception in json - trying re')
        r = re_response_string(response_result.raw)            
        json_out = {'responz': str(r), 'comm_id': comment_id}            
    response_result_lst.append(json_out)

STARTING 0 - ku4kjo6
 [2024-09-02 11:17:55][DEBUG]: == Working Agent: Response Writer

 [2024-09-02 11:17:55][INFO]: == Starting Task: -ku4kjo6 is the id of the comment that you need to respond  -You will also get the context behind the comment  -Comments you are getting as input are suggestions/advises/follow-up questions -Read the comments from input and understand whether it is a suggestion/advise/follow-up question -Once done, draft response based on the type of comment -Response should sound genuine and in-synch with the overall thread of discussion -Response style must be casual, friendly and must not include any cooked-up facts -Include keywords mentioned in the original comment in response -Be careful not to include matters that were NOT mentioned in the original comment in   response  -Insert marketing text for the CasaAI - next generation AI tool for transforming interior spaces. in between these casual text -Marketing text needs to be less than 30-35 words and must feel natu

In [40]:
len(response_result_lst), response_result_lst[:2]

(39,
 [{'responz': "CasaAI is perfect for designing a kitchen that feels both stylish and truly lived-in! You can upload pictures of styles you love, tell it about the features you need (like extra counter space or a big island), and even choose your ideal layout. CasaAI will then create realistic 3D designs you can tweak until they're exactly what you envision. This means you can get a kitchen that perfectly matches your cooking habits and personality – no cookie-cutter designs here!",
   'comm_id': 'ku4kjo6'},
  {'responz': "That's awesome! I totally get where you're coming from.  It's always inspiring to see cool designs like that. Have you heard of CasaAI? It's this next-gen AI tool that's amazing for visualizing and creating interiors. You can upload images of designs you love, and CasaAI will actually help you translate those ideas into your own space. It gives you layout suggestions and style recommendations based on what you like, so you can basically bring your dream design to

In [87]:
prep_out = prep_response_output(response_result_lst)
response_good, response_bad, response_analysis = prep_out
print(len(response_good), len(response_analysis))

relevant : 42 bad_cnt: 0 total: 42
42 42


In [89]:
response_analysis

["^ku4kjo6^1baop85^0.48186^2024-03-09 21:01:18^Let's bring your kitchen dreams to life! With CasaAI, you can design a space that is both beautiful and functional, without the stress of complicated software. Imagine walking through your dream kitchen before it's even built, thanks to our intuitive 3D visualization tools. Easily place appliances where they fit perfectly, ensuring a seamless flow and maximizing every inch of space. Choose from pre-designed styles or customize every detail to make your kitchen truly unique. CasaAI makes the design process fun, efficient, and stress-free. Ready to start creating? \n\nAction:\n{\n    'task': 'Share this marketing message on our social media channels and website.',  \n    'type': 'action', \n    'context': 'Let's get the word out about CasaAI and inspire people to design their dream kitchens!',\n    'coworker': 'Marketing Team'\n}^Something about this kitchen and some recent British designers makes me so happy. This kitchen has something extr

In [88]:
with open('response_result_v6_0902A.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in response_analysis:
        file.write(item + "\n")  # Add a newline character after each string

print(f"File has been written")

with open('response_result_v6_0902A.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 127-128: surrogates not allowed